<a href="https://colab.research.google.com/github/WetSuiteLeiden/data-collection/blob/master/gemeentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# (only) in colab, run this first to install wetsuite from (the most recent) source. 
#    (this should soon simplify to something like   !pip3 install --upgrade wetsuite)
# For your own setup, see wetsuite's install guidelines.
!pip3 install -U wetsuite

# Purpose of this notebook

Demonistrates how we fetch some information about municipalities

Can't really be considered a dataset, but is a little more than a list.

Roughly:
- fetching an existing gemeente-metaata CSV from overheid.nl, 
- extracting only some of the more interesting data,
- make it into something more digestible in python.
- ALSO: fetch some organisational relationships from a semantic store that is mentioned in one of the fields

CONSIDER: 
- try to figure out gemeentes that no longer exist, e.g. from old CVDR entries, and/or [wikipedia](https://nl.wikipedia.org/wiki/Lijst_van_voormalige_Nederlandse_gemeenten)
- broaden this, to e.g. include the codes for the provincies and waterschappen    https://tardis.overheid.nl/waardelijsten.html


It was made to support searches, e.g. a "search CVDR for each gemeente individually",
and could be used for tother things like looking up city-gemeente relationships should you need it.

It is also potential demonstation of some "it's not that hard to fetch and combine data from the internet, if you don't make life too hard on yourself".

The combination of the CSV and semantic fetches should take a few minutes overall.

## Open questions

* What happens in other datasets when municipalities merge?
  * Presumably existing documents's metadata does not get edited, so most current and all old documents will not be found under the current name?

* Where might we get historical data about these mergers?

## The actual fetching

In [1]:
import pprint, datetime, pprint, json, io, csv, re

import rdflib # if not installed, do a   !pip install rdflib
from rdflib.namespace import RDF, RDFS, SKOS, FOAF, DCTERMS

import wetsuite.helpers.etree
import wetsuite.helpers.net
import wetsuite.helpers.strings

In [3]:
# definitions an helpers we'll use later

OVERHEID = rdflib.Namespace('http://standaarden.overheid.nl/owms/terms/')

_fetchcache = {}
def urlfetch(url):
    " Fetch data from URL.  Caches results in memory, to avoid fetching the same thing multiple times within a run of this script "
    if url in _fetchcache:
        #print( 'CACHED', url )
        return _fetchcache[url]
    else:
        print( 'FETCHING', url )
        #CONSIDER: localdata cache -  faster during debug, but requires that database to be set up (and wouldn't fetch the newest values)
        data = wetsuite.helpers.net.download( url ) # could also use urllib to make it more independent.
        _fetchcache[url] = data
        return data


def labels_from_n3(url):
    ' fetch n3 from URL, parse, then pick out and return _just_ the label '
    g = rdflib.Graph()
    n3data = urlfetch( url )
    graph = g.parse(data=n3data, format="n3")
    ret = []
    for a,b,c in graph:
        if 'http://www.w3.org/2000/01/rdf-schema#label' in b:
            ret.append( c.value )
    return tuple(ret)

In [4]:
## Fetch and parse the CVS that is most of this

class GemeenteCSVDialect(csv.Dialect):
    delimiter      = ';'
    doublequote    = True
    quotechar      = '"'
    quoting        = csv.QUOTE_ALL
    lineterminator = '\n'
    header         = True

#csv.field_size_limit(1000000)

# Used to be utf-16, and take ~15s to fetch.   It seems this the slowness was fixed, and the encoding became utf8, somewhere late 2023
csv_bytes  = urlfetch( 'https://organisaties.overheid.nl/export/Gemeenten.csv' )
csv_utext  = csv_bytes.decode('utf8') 
csv_parsed = csv.DictReader( io.StringIO(csv_utext) , dialect=GemeenteCSVDialect())  # is a generator, seems to parse as it goes

FETCHING https://organisaties.overheid.nl/export/Gemeenten.csv


In [5]:
gemeentes_overheidnl = []  # list of dicts

for row_dict in list( csv_parsed ): # yields just the data rows, as a {headerkey:value} dict
    print( row_dict )
    selected_dict = {}
    for field_name, value in row_dict.items():
        # - removes fields that we know are less interesting  or just too sparesely filled
        #   keep in mind that these seem to change over time.
        # - renames some long-named fields
        
        ## Currently ignored, might use later
        if wetsuite.helpers.strings.contains_all_of(field_name, ('Adressen', 'type', 'toelichting') ): 
            continue # not really interested, though
            field_name = 'Adressen'
            new_value = []
            for part in value.split(','):
                k, v = part.split(':', 1)
                new_value.append( (k.strip(), v.strip()) )
            value = new_value


        ## Known things to ignore
        elif wetsuite.helpers.strings.contains_any_of( field_name, (  
            'Classificaties','E-mail', 'Internet', 'Telefoon', 'Fax', 'Bezoekadres',  'Postadres',   
            'Contact pagina', 'Sociale Media', "Contactpagina's",
            'Actieve afwijkingen van de Regeling', 'Relatie met ministerie', 'Beleidsterreinen',  
            'Startdatum', 'Einddatum',
            'Type', # should always be Gemeente, in this list at least
            'Subtype',
            'Alternatieve naam', # seems empty
            'Inwoners per km2', # we have both inwoners and km2 separately anyway
            'Beschrijving', 'Informatie Woo-verzoek', 'OIN',  'Loonheffingennummer', 'btw-nummer', 'KVK-nummer',
            'Datum van publiceren begroting', 
            'Wateroppervlakte',
            'Onderdeel van',
            'Datum ter verificatie',
            'Laatste mutatie',
            'rganisatiebeschrijving',
            'Online afspraak', 'Afspraak per', 'Telefonische afspraak',
            'organogram', 
            #'Laatste mutatie'
            'TOOI URI', # TODO: figure out what exactly we can do with the TOOI and OWMS links

        ) ):
            continue


        ## Known things to pass through, as-is or with some tweaking
        elif field_name in ('Afkorting', 'Aantal inwoners', 'Totaal aantal zetels', 'Organisatiecode', 'Officiële naam' ):
            pass #as-is

        elif wetsuite.helpers.strings.contains_all_of(field_name, ('Raad', 'Partij', 'aantal zetels') ):
            field_name = 'Raad'
            new_value = []
            for s in value.split(';'):
                m = re.match('(.+) ([0-9]+)$', s.strip())
                if m is not None:
                    name, count = m.groups()
                    print( repr( [s, name, count.strip('() ') ] ) )
                    new_value.append( (name, int(count.strip('() '))) )
            value = new_value

        elif field_name == 'Bevat plaatsen':
            value = list( s.strip()  for s in value.split(',') )

        elif field_name == 'Oppervlakte (km2)':
            field_name = 'Oppervlakte'
            try:
                value = (int(value.rsplit(',',1)[0]), 'km2')    # that comma is confusable
            except:
                value = ''

        elif field_name == 'Adressen':
            value = list( s.strip() for s in value.split(',') )
            continue

        elif field_name == 'OWMS URI':  # on top of passing that URL though, fetch some details from the (indirectly) referenced RDF
            value = value.replace("'", '%27') # the URL is either given to us decoded, or never encoded in the first place, but either way...
            if 1: # was temporarily disabled while there was a bug in the site
                service_area, overlaps, predecessors = set(), set(), set()
                n3_url = '%s.n3'%(value)
                if len(n3_url) > 10: #deals with one missing case
                    g = rdflib.Graph()
                    n3data = urlfetch( n3_url )
                    graph = g.parse(data=n3data, format="n3")
                    for a,b,c in graph: # note that c is often a  rdflib.term.Literal (hence the .value)  or a rdflib.term.URIRef (which here is coerced to str)
                        if b == OVERHEID.serviceAreaOf:
                            service_area.add( labels_from_n3( c + '.n3' ) )
                        elif b == OVERHEID.hasServiceArea:
                            pass
                        elif b == OVERHEID.CBSCode:
                            selected_dict['CBSCode'] = c.value
                        elif b == SKOS.altLabel:
                            selected_dict['altLabel'] = c.value
                        elif b == SKOS.prefLabel:
                            selected_dict['prefLabel'] = c.value
                        elif b == OVERHEID.overlapsWith:
                            overlaps.add(     labels_from_n3( c + '.n3' ) )
                        elif b == OVERHEID.predecessor: # previously in gemeente...
                            predecessors.add( labels_from_n3( c + '.n3' ) )

                        # CONSIDER: maybe use
                        elif b == RDFS.label:
                            pass
                        elif b == SKOS.prefLabel:
                            pass
                        elif b == OVERHEID.replaces:
                            pass
                        elif b == OVERHEID.successor: 
                            pass
                        elif b == OVERHEID.grondslag:
                            pass

                        # probably not
                        elif b in (RDFS.isDefinedBy, SKOS.note, SKOS.changeNote, FOAF.page, OVERHEID.startDate, OVERHEID.endDate ):
                            pass
                        elif b == RDF.type:
                            pass
                        elif b  in (DCTERMS.publisher, DCTERMS.rights, DCTERMS.isPartOf, DCTERMS.date):
                            pass
                        
                        else:
                            print('UNKNOWN RELATION', b,c)
                    
                    selected_dict['Service area of'] = sorted(  service_area )
                    selected_dict['Overlaps with']   = sorted(  overlaps     )
                    selected_dict['Predecessors']    = sorted(  predecessors )

        else:
            print("UNKNOWN field %r (%r)"%(field_name,value))
        
        selected_dict[ field_name ] = value

    #take a few sources of names and put them into one field, in order of most useful
    selected_dict['Namen'] = wetsuite.helpers.strings.ordered_unique(
        list( selected_dict.pop(key, None)   for key in ('Afkorting', 'Officiële naam', 'altLabel', 'prefLabel') ),
        remove_none=True
    )

    gemeentes_overheidnl.append( selected_dict )

{'Officiële naam': 'Gemeente Aa en Hunze', 'Alternatieve naam': '', 'Afkorting': 'Aa en Hunze', 'Type': 'Gemeente', 'Subtype': '', 'Startdatum': '01-01-1998', 'Einddatum': '', 'Datum ter verificatie': '22-07-2024', 'Laatste mutatie': '22-07-2024', 'Classificaties': 'CAO, wettelijke-grondslagen: https://www.caogemeenten.nl/, opschrift: CAO Gemeenten;Overheidswerkgever, externe-url: https://www.wnra.nl/, wettelijke-grondslagen: https://wetten.overheid.nl/BWBR0001947, opschrift: Ambtenarenwet 2017;WNT-instelling, externe-url: https://www.topinkomens.nl/voor-wnt-instellingen/wnt-register, wettelijke-grondslagen: http://wetten.overheid.nl/jci1.3:c:BWBR0032249&paragraaf=1&artikel=1.2&lid=1, opschrift: Gemeenten (Artikel 1.2, lid 1c, WNT);Woo, externe-url: https://www.rijksoverheid.nl/onderwerpen/wet-open-overheid-woo, wettelijke-grondslagen: https://wetten.overheid.nl/BWBR0045754, opschrift: Wet open overheid', 'Onderdeel van': '', 'Organisatiebeschrijving/doel': '', 'Link naar uitgebreidere

In [6]:
#pprint.pprint( gemeentes_overheidnl )
gemeentes_overheidnl

[{'Oppervlakte': (279, 'km2'),
  'Bevat plaatsen': ['Amen',
   'Anderen',
   'Anloo',
   'Annen',
   'Annerveenschekanaal',
   'Balloerveld',
   'Balloo',
   'Deurze',
   'Eext',
   'Eexterveen',
   'Eexterveenschekanaal',
   'Eexterzandvoort',
   'Ekehaar',
   'Eldersloo',
   'Eleveld',
   'Gasselte',
   'Gasselternijveen',
   'Gasselternijveenschemond',
   'Gasteren',
   'Geelbroek',
   'Gieten',
   'Gieterveen',
   'Grolloo',
   'Marwijksoord',
   'Nieuw Annerveen',
   'Nieuwediep',
   'Nijlande',
   'Nooitgedacht',
   'Oud Annerveen',
   'Papenvoort',
   'Rolde',
   'Schipborg',
   'Schoonloo',
   'Spijkerboor DR',
   'Vredenheim'],
  'Aantal inwoners': '25845',
  'Totaal aantal zetels': '21',
  'Raad': [],
  'CBSCode': '1680',
  'Service area of': [('Alescon',),
   ('GGD Drenthe', '90'),
   ('Regiokorps Politie Drenthe',),
   ('Regionale Uitvoeringsdienst Drenthe', 'RUD Drenthe'),
   ('Samenwerkingsorganisatie Publiek Vervoer Groningen Drenthe',),
   ('VRD', 'Veiligheidsregio Dren

In [7]:
## Write that data structure to a file
today_str = datetime.date.today().strftime('%Y-%m-%d')
with open('gemeentes-struc.json', 'wb') as f:
    dataset = {
        'description_short':'A list of gemeentes, and some details about each.',
        'description':'''
This is largely the more interesting fields from https://organisaties.overheid.nl/export/Gemeenten.csv
augmented with some of the RDF data like that under https://standaarden.overheid.nl/owms/terms/Leiden_(gemeente)


.data is a list of dicts, one per gemeente (currently 344 of them). Keys in that dict include:

'Namen' - a list of name variants. 
    Usually just the short name, and a longer one with "Gemeente " in front
    Sometimes with alternative names, e.g. ["Den Bosch", "Gemeente 's-Hertogenbosch", "'s-Hertogenbosch"]
    We have used these as "Match one of these" to search for gemeentebeleid per gemeente

Descriptions like 'Aantal inwoners', 'Oppervlakte'

Organisational relations like 
    - 'Bevat plaatsen'
    - 'Overlaps with', mentioning Provinces, Waterschappen
    - 'Service area of' - things like GGD, Police, Social services  (each item is a list because we tend to have a full name and an abbreviation)
    - 'Predecessors' - mentions previous gemeente names

Useful in other APIs / joining with other data
- 'OWMS URI'
- 'CBSCode', 'Organisatiecode'

Politics
- 'Totaal aantal zetels', 'Raad'


This version was fetched from overheid.nl on %s
        '''%today_str,
        'fetched':today_str, 
        'data':gemeentes_overheidnl,
    }

    f.write( json.dumps( dataset ).encode('u8') )

In [ ]:
# # from wikipedia
# import bs4, requests, urllib.parse 
# import wetsuite.helpers.net
# url = 'https://nl.wikipedia.org/wiki/Lijst_van_Nederlandse_gemeenten'
# soup = bs4.BeautifulSoup( wetsuite.helpers.net.download( url ), features="xml" )
# rows = soup.select("#mw-content-text table.wikitable tbody tr")
# gemeentes_wiki = []
# for row in rows:
#     ths = row.select('th')
#     if len(ths) == 1: # data rows use oe TH for the name, easiest way to filter out the header rows
#         cells = row.select('td, th')
#         if len(cells) in (10,11): # real data row, last column may be missing
#             name_flag, cbscode, province, population, area_km2, _, _,_,_, kaartimg, inkomen = (cells+[None]*4)[:11]
#             name = name_flag.text.strip() # CONSIDER: URL to flag, URL to map image
#             if name in ('Bonaire', 'Sint Eustatius', 'Saba' ):
#                 continue
#             #flaglink = name_flag.find('a')
#             #if flaglink is not None:
#             #    print( urllib.parse.urljoin( url, flaglink.find('img').get('src') ) ) # flag url. These seem to be effectively public domain (see the link)
#             cbscode = cbscode.text.strip()
#             province = province.text.strip()
#             population =   int( float( population.text.replace('.','') ) )
#             area_km2   = float( area_km2.text.replace(',','.') )
#
#             gemeentes_wiki.append( (name, cbscode, province, population) )
#         else:
#             raise ValueError( 'weird row (%d): %r'%(len(cells), row) )
#
#
# pprint.pprint(gemeentes_wiki)

## 